In [29]:
import os

import mxnet as mx
from mxnet.gluon.block import HybridBlock
from mxnet.gluon import nn
from mxnet.gluon.model_zoo import model_store
from mxnet import autograd

import numpy as np
from relu_noneg import Relu

import cv2

In [6]:
class AlexNet(HybridBlock):
    def __init__(self, classes=1000, **kwargs):
        super(AlexNet, self).__init__(**kwargs)
        with self.name_scope():
            self.features = nn.HybridSequential(prefix='')
            with self.features.name_scope():
                self.features.add(nn.Conv2D(64, kernel_size=11, strides=4, padding=2))
                self.features.add(Relu())
                self.features.add(nn.MaxPool2D(pool_size=3, strides=2))
                
                self.features.add(nn.Conv2D(192, kernel_size=5, padding=2))
                self.features.add(Relu())
                self.features.add(nn.MaxPool2D(pool_size=3, strides=2))
                
                self.features.add(nn.Conv2D(384, kernel_size=3, padding=1))
                self.features.add(Relu())
                
                self.features.add(nn.Conv2D(256, kernel_size=3, padding=1))
                self.features.add(Relu())
                
                self.features.add(nn.Conv2D(256, kernel_size=3, padding=1))
                self.features.add(Relu())
                self.features.add(nn.MaxPool2D(pool_size=3, strides=2))
                
                self.features.add(nn.Flatten())

                self.features.add(nn.Dense(4096))
                self.features.add(Relu())
                self.features.add(nn.Dropout(0.5))

                self.features.add(nn.Dense(4096))
                self.features.add(Relu())
                self.features.add(nn.Dropout(0.5))

            self.output = nn.Dense(classes)

    def hybrid_forward(self, F, x):
        x = self.features(x)
        x = self.output(x)
        return x

# Constructor
def alexnet(pretrained=False, ctx=mx.cpu(),
            root=os.path.join('~', '.mxnet', 'models'), **kwargs):
    r"""AlexNet model from the `"One weird trick..." <https://arxiv.org/abs/1404.5997>`_ paper.

    Parameters
    ----------
    pretrained : bool, default False
        Whether to load the pretrained weights for model.
    ctx : Context, default CPU
        The context in which to load the pretrained weights.
    root : str, default '~/.mxnet/models'
        Location for keeping the model parameters.
    """
    net = AlexNet(**kwargs)
    if pretrained:
        net.load_params(model_store.get_model_file('alexnet', root=root), ctx=ctx)
    return net

In [7]:
alexnet = alexnet(pretrained=True)

In [8]:
def preprocess(data):
    data = mx.image.imresize(data, 256, 256)
    data, _ = mx.image.center_crop(data, (224, 224))
    data = data.astype(np.float32)
    data = data/255
    data = mx.image.color_normalize(data,
                                    mean=mx.nd.array([0.485, 0.456, 0.406]),
                                    std=mx.nd.array([0.229, 0.224, 0.225]))
    data = mx.nd.transpose(data, (2,0,1))
    return data

In [23]:
with open("img/snake.jpg", 'rb') as fp:
    str_image = fp.read()

image = mx.img.imdecode(str_image)
image = preprocess(image)
image = image.expand_dims(axis=0)

out = alexnet(image)
out.argmax(axis=1)


[ 56.]
<NDArray 1 @cpu(0)>

In [ ]:
for k, v in alexnet.collect_params().items():
    print(k)
    print(v.grad().shape)

In [24]:
image.attach_grad()
with autograd.record():
    out = alexnet(image)

In [25]:
out.backward()
image.grad.shape

(1, 3, 224, 224)

In [26]:
def save_gradient_images(gradient, file_name):
    """
        Exports the original gradient image
    Args:
        gradient (np arr): Numpy array of the gradient with shape (3, 224, 224)
        file_name (str): File name to be exported
    """
#     if not os.path.exists('../results'):
#         os.makedirs('../results')
    gradient = gradient - gradient.min()
    gradient /= gradient.max()
    gradient = np.uint8(gradient * 255).transpose(1, 2, 0)
#    path_to_file = os.path.join('../results', file_name + '.jpg')
    path_to_file = file_name + '.jpg'
    # Convert RBG to GBR
    gradient = gradient[..., ::-1]
    cv2.imwrite(path_to_file, gradient)


In [34]:
guided_grads = image.grad[0].asnumpy()
save_gradient_images(guided_grads, "test")

In [35]:
def convert_to_grayscale(cv2im):
    """
        Converts 3d image to grayscale
    Args:
        cv2im (numpy arr): RGB image with shape (D,W,H)
    returns:
        grayscale_im (numpy_arr): Grayscale image with shape (1,W,D)
    """
    grayscale_im = np.sum(np.abs(cv2im), axis=0)
    im_max = np.percentile(grayscale_im, 99)
    im_min = np.min(grayscale_im)
    grayscale_im = (np.clip((grayscale_im - im_min) / (im_max - im_min), 0, 1))
    grayscale_im = np.expand_dims(grayscale_im, axis=0)
    return grayscale_im

In [36]:
grayscale_guided_grads = convert_to_grayscale(guided_grads)

In [37]:
save_gradient_images(grayscale_guided_grads, "gray")

In [38]:
def get_positive_negative_saliency(gradient):
    """
        Generates positive and negative saliency maps based on the gradient
    Args:
        gradient (numpy arr): Gradient of the operation to visualize
    returns:
        pos_saliency ( )
    """
    pos_saliency = (np.maximum(0, gradient) / gradient.max())
    neg_saliency = (np.maximum(0, -gradient) / -gradient.min())
    return pos_saliency, neg_saliency

pos_sal, neg_sal = get_positive_negative_saliency(guided_grads)
save_gradient_images(pos_sal, "pos_sal")
save_gradient_images(neg_sal, "neg_sal")